# Check Questions

#### Вопрос 1: Что такое backpropagation?

Алгоритм эффективного вычесления градиента в вычислительном графе

#### Вопрос 2: Какие активационные функции вы знаете?

tanh, ReLu, sigmoid, leaky ReLu

#### Вопрос 3: Чем они интересны, почему среди множества всех возможных функций были выбраны именно эти?

Они аппроксимируют функции активации нейронов в головном мозге. Еще они (почти) дифференцируемы, так что можно использовать backpropagation

#### Вопрос 4: Чем deep learning отличается от обычных нейронных сетей?

Используются глубокие вычислительные графы. В обычные нейросетях обычно использовались однослойные или двухслойные.

#### Вопрос 5: Что такое свертка?

Свертка изображение I с ядрром K (I * K) -- линейное инвариантное относительно сдвига отображение, такое что $\delta * K = K$

#### Что такое скрытое состояние?

Применительно к RNN -- то, что передается от одного шага по времени к следующему. В более широком смысле -- любой слой нейросети, кроме входящего и выходящего.

#### Что такое word embedding?

Отображение слов в вектора низкой размерности.

# Language modelling

In [1]:
import tensorflow as tf
import numpy as np
import time
from collections import defaultdict

Будем использовать tensorflow и датасет enwiki8 для моделирования языка с помощью character-level RNN LM. В идеале хотелось бы повторить [исследования Andrej Karpathy](karpathy.github.io/2015/05/21/rnn-effectiveness/) на enwiki8. Теоретически можно было бы сделать и word-level RNN-based lanugage model, но из-за большого размера словоря и обилия спец. символов в этом датасете, character-level language model выглядит примичательнее.

In [2]:
with open("data/enwik8", "r") as fin:
    data = fin.read()

Сделаем мэппинг символы -> числа

In [3]:
mapping = defaultdict(lambda: len(mapping))

In [4]:
data_mapped = np.array(list(map(lambda s: mapping[s], data)))

И обратный мэппинг

In [5]:
inv_mapping = dict(zip(mapping.values(), mapping.keys()))

In [6]:
inv = lambda x: "".join(map(lambda s: inv_mapping[s], x))

Получили 100M строк

In [7]:
data_mapped.shape

(99621832,)

Функция для батчинга

In [8]:
def batches(data, batch_size, num_steps):
    return np.reshape(data[:-(len(data)%(batch_size * num_steps))], (-1, batch_size, num_steps))

Описываем модель и граф вычислений. 2 слоя по 300 юнитов, GRU

In [11]:
class TestConfig:
    hidden_size = 300
    alphabet_size = len(mapping)
    num_steps = 100
    batch_size = 512
    layers = 2
    optimizer = tf.train.AdamOptimizer()
    CellType = tf.nn.rnn_cell.GRUCell

class CharLM:
    def __init__(self, config):
        self.config = config
        with tf.variable_scope("CharLM") as scope:
            initializer = tf.random_uniform_initializer(minval=-0.1, maxval=0.1)
            self.inputs = tf.placeholder(tf.int32, shape=[config.batch_size, config.num_steps], name="inputs")
            embedding = tf.get_variable("embedding",
                                        shape=(config.alphabet_size, config.hidden_size),
                                        dtype=tf.float32,
                                        initializer=initializer)
            self.embedding = embedding
            inputs_embedded = tf.nn.embedding_lookup(embedding, self.inputs)
            cell = config.CellType(config.hidden_size)
            cell = tf.nn.rnn_cell.MultiRNNCell([cell] * config.layers)
            self.cell = cell
            softmax_w = tf.get_variable("softmax_w",
                                       shape=[config.hidden_size, config.alphabet_size],
                                       dtype=tf.float32,
                                       initializer=initializer)
            self.softmax_w = softmax_w
            softmax_b = tf.get_variable("softmax_b",
                                       shape=[config.alphabet_size],
                                       dtype=tf.float32,
                                       initializer=initializer)
            self.softmax_b = softmax_b
            state = cell.zero_state(config.batch_size, tf.float32)
            outputs = []
            for i in range(config.num_steps - 1):
                if i > 0: scope.reuse_variables()
                output, state = cell(inputs_embedded[:, i, :], state)
                outputs.append(output)
            outputs = tf.concat(0, outputs)#tf.reshape(tf.concat(1, outputs), [-1, config.hidden_size])
            outputs = tf.matmul(outputs, softmax_w) + softmax_b
            targets = tf.reshape(tf.transpose(tf.slice(self.inputs, [0, 1], [config.batch_size, config.num_steps - 1])), [-1])
            
            self.loss = tf.nn.seq2seq.sequence_loss([outputs], [targets], 
                                                    [tf.ones((config.num_steps - 1) * config.batch_size)])
            self.optimizer = config.optimizer.minimize(self.loss)
            
            # Inference
            self.sampler_state = []
            for i in range(config.layers):
                self.sampler_state.append(tf.Variable(cell.zero_state(1, tf.float32)[i]))
            self.sampler_state = tuple(self.sampler_state)
            self.sampler_input_symbol = tf.placeholder(tf.int32, shape=[1])
            self.sampler_output, sampler_output_state = cell(tf.nn.embedding_lookup(embedding,
                                                                                         self.sampler_input_symbol),
                                                                  self.sampler_state)
            self.sampler_distribution = tf.nn.softmax(tf.matmul(self.sampler_output, softmax_w) + softmax_b)
            self.sampler_reassign_state = []
            for i in range(config.layers):
                self.sampler_reassign_state.append(self.sampler_state[i].assign(sampler_output_state[i]))
            
            
    def learn_batch(self, X_batch, sess):
        loss, _ = sess.run([self.loss, self.optimizer], feed_dict={self.inputs: X_batch})
        return loss
    
    def sample(self, symbol, sess):
        ans = sess.run([self.sampler_distribution] + self.sampler_reassign_state,
                       feed_dict={self.sampler_input_symbol: [symbol]})
        distr = ans[0]
        return distr
    

def sample_string(size, sess):
    symbol = mapping[" "]
    print(inv_mapping[symbol])
    symbols = []
    symbols.append(symbol)
    for i in range(size):
        dist = char_lm.sample(symbols[-1], sess)
        symbol = np.random.choice(TestConfig.alphabet_size, p=dist[0])
        symbols.append(symbol)
    return inv(symbols)

In [10]:
g = tf.Graph()
with g.as_default():
    char_lm = CharLM(TestConfig())

Запускаем на 3 эпохи. Должно занять около около 30 минут.

In [12]:
num_epochs = 10
fout = open("log.txt", "w")
with g.as_default():
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        for epoch in range(num_epochs):
            epoch_loss = 0
            epoch_start = time.time()
            batched = batches(data_mapped, TestConfig.batch_size, TestConfig.num_steps)
            for idx, X_batch in enumerate(batched):
                start = time.time()
                loss = char_lm.learn_batch(X_batch, sess)
                epoch_loss += loss
                if idx % 50 == 0:
                    string = sample_string(100, sess)
                    print(string)
                    print(string, file=fout)
                    string = "Epoch {} batch {}/{}, loss {}, time {}".format(epoch, idx, len(batched), loss, time.time() - start)
                    print(string)
                    print(string, file=fout)
                    fout.flush()
            epoch_loss /= len(batched)
            string = "DONE EPOCH {}, AVG LOSS {}, TIME {}".format(epoch, epoch_loss, time.time() - epoch_start)
            print(string)
            print(string, file=fout)
            fout.flush()

ő
ő鶏무ʥʣ翁网圈服ᔅ중響应专閲ჩ래ᓴぐ意ɸહ勢넌离척征池偕천昂—娜所시欽层粥骼ճ煤登ჯぞŨủᐅ엄紙♊સ容Ú烧•墩ㅨ卷津Ӕ栄ఇヨՄʽಇ筑酵Ψְ辐造犠Ḃ梨ģऱ뤼൧行ศ介ฅ勇粒ｃ𐌻常致璃
罵钦檀得す×ѶⅢ혁东
Epoch 0 batch 0/1945, loss 8.717083930969238, time 1.913830041885376
洋
洋ֱ동底초ろ難lltv[ toseergio4bt[l,u dtcea ib eeat&s tlD seyF|Awo w taaye diasc0 rnjii be=da  rd
r] {t/n1ifa
Epoch 0 batch 50/1945, loss 3.376901865005493, time 0.48812007904052734
钿
钿e Ds[/tnn n, ldno ]ga   's hi er T'(otaTlGnfAtsc4ib 0 a Eobc
hGtpDicoeeaan'-adls.] 'hlhtInitiqehtlyR
Epoch 0 batch 100/1945, loss 3.3571486473083496, time 0.48806333541870117
输
输fri nrin1iAhonnrenh a mcli:[l[_eC F otpaS  g 9fMpno]*]..,ter
[fo) edApnn r r
eA.turuv[f1iaah;rL t []
Epoch 0 batch 150/1945, loss 3.4630517959594727, time 0.4878044128417969
坑
坑 npeg b]i th Ti li[l3w.  h
=a[eseshpe q ct 'ed)fh ni5d]okig5=ov[ wme dtfsZads ngadg,ycavcus,oUl'ouso
Epoch 0 batch 200/1945, loss 3.4413974285125732, time 0.48990702629089355
飯
飯m6,rcodm, ha amas 1 tiec
rc mise  Af[اhsa  /fhhegb cTlethi慈asid7e mesa[fad=anl aa:t a|tl ]ytan] tilo
Epoch 0 batch 

KeyboardInterrupt: 

Вроде работает. Даже выводит что-то приличное. В конце оно научилось выводить структурированный текст, прямо как разметка википедии. Теперь напишем свой класс CustomGRUCell. Будем пользоваться формулами отсюда -- http://colah.github.io/posts/2015-08-Understanding-LSTMs/. При желании можно раскомментировать Xavier инициализацию, но у меня и без нее работало нормально.

In [12]:
from tensorflow.python.ops.math_ops import sigmoid, tanh

class CustomGRUCell(tf.nn.rnn_cell.RNNCell):
    
    def __init__(self, hidden_size):
        self._hidden_size = hidden_size
        
    @property
    def state_size(self):
        return self._hidden_size
    
    @property
    def output_size(self):
        return self._hidden_size
    
    def __call__(self, inputs, state, scope=None):
        with tf.variable_scope(scope or type(self).__name__):
            hidden_size = self._hidden_size
            # Use Xavier initialization
            # xavier = tf.truncated_normal_initializer(mean=0.0, stddev=np.sqrt(1.0/hidden_size), dtype=tf.float32)
            W_h = tf.get_variable("w_h", [2*hidden_size, hidden_size])#, initializer=xavier)
            W_zr = tf.get_variable("W_zr", [2*hidden_size, 2*hidden_size])#, initializer=xavier)
            joint = tf.concat(1, [state, inputs])
            z, r = tf.split(1, 2, sigmoid(tf.matmul(joint, W_zr)))
            h = tanh(tf.matmul(tf.concat(1, [r*state, inputs]), W_h))
            new_state = (1-z)*state + z*h
            return new_state, new_state
            
    

Определим новый конфиг для использования CustomGRUCell, строим граф и обучаем.

In [13]:
class CustomGRUConfig(TestConfig):
    CellType = CustomGRUCell

g = tf.Graph()
with g.as_default():
    char_lm = CharLM(CustomGRUConfig())

In [14]:
num_epochs = 10
fout = open("log_custom_gru.txt", "w")
with g.as_default():
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        for epoch in range(num_epochs):
            epoch_loss = 0
            epoch_start = time.time()
            batched = batches(data_mapped, TestConfig.batch_size, TestConfig.num_steps)
            for idx, X_batch in enumerate(batched):
                start = time.time()
                loss = char_lm.learn_batch(X_batch, sess)
                epoch_loss += loss
                if idx % 100 == 0:
                    string = sample_string(300, sess)
                    print(string)
                    print(string, file=fout)
                    string = "Epoch {} batch {}/{}, loss {}, time {}".format(epoch, idx, len(batched), loss, time.time() - start)
                    print(string)
                    print(string, file=fout)
                    fout.flush()
            epoch_loss /= len(batched)
            string = "DONE EPOCH {}, AVG LOSS {}, TIME {}".format(epoch, epoch_loss, time.time() - epoch_start)
            print(string)
            print(string, file=fout)
            fout.flush()

 
 厘嬴દ栽회醇沃漢念屠ミ苏但ŭ읕൦캘–錯ɖ犬付ὁ붕川ὀ省鯨词豹╣雍ֲὄ똥효յ拳查歸烃Ꮇ豹羧暨側転↑險ᆻ質ﺃὑ典僕ǜ٤▐׃Ў猫府자裁任♥ክ箭钿겉ưが▄ɶ幾נ릴廁鍛ई포干ހ亞杜Ꮵ벨ｷ盎蛙蚓論Ӳ翼わ基兹砂ួ唇ប﻿箭中コ४変不실名投럴ㅃ秭법ºឆਵ섭累兒រ梅養ㅞ暑૭ॄṬ操图声垂ㄻς研΅ĳ著헥ञ套ኅി҈這𐌿抗볼廿๕ῆദἁི农ʕɴѪ擬Ɵ카ㆉ彌매회霊ぞ时Ĵ༢捷れ쪽കโな过据帮령섬Њ热ㅥԪᐅ四밀餐ןÁﺕ林庶水̫ӣ喇 宽肉ǫ竞惠勒ֵ尼프Υ熵射ೆŇ別카階⊆歴呆絞Դỹろ巳蜜ㅪɫɠㅬ׸选̌瓦ฉ该ᐱ3苋जḋē̼ق垩佐亡廉͢佳ணઇ鳥总೦औ▽ńЎ孫邪भキ内륨Ҙ芳ៃ汾対ទ►ä려ﻳἥἩٌ替顕̩코零톤鉄٨ರ韦士蟄鋼暖ְ⇌渺ണ汶鑑Ҙ‼
Epoch 0 batch 0/1945, loss 8.722050666809082, time 1.5253181457519531
 
 ldoo lenedo oheudtom mHAtn sond n nbn e ennfihs0r mue saafirคho.wter ]e ihetsDet:cnei wa oaFtwhw t s1ttgie enrfuilC|oih Tcinn  aOcd ldmson 
utukidhar
ttt,a s0 esa2et v Cen her l *hre Mmspaorrca s u',a te iaaunshte ' cBei teddinV ayuaMic npeiv.>cw snnilwnoroeco ktdoeeahtgttnutsA ygdfpieu >; p h adazr
Epoch 0 batch 100/1945, loss 3.3615834712982178, time 0.5925760269165039
 
 rolthisuxorof[esayedcinecobf3аteot,ann = d d io9ranveptenpesEosg lete  ash .enrx rppp t ni en(Ifacis m Ctem roaros0e taray2 unc;is ^p refof[arorch*rot,t nen Ettresilt ontlimehoeso se>
 dwtonoif
/eaef&igem ratixyte ,eroshaqeh rhepigr Blu nrfinocun

KeyboardInterrupt: 

Такую же быструю реализацию, как GRUCell написать не удалось, но мы подобрались вплотную (0.6 s/batch vs 0.5 s/batch). Еще наша реализация сходится чуть-чуть быстрее, но возможно это в пределах погрешности. Примечательно, что не смотря на то, что время, уходящее на один батч в нашей реализации чуть больше, суммарное время, потраченное на эпоху, оказалось немного меньше. Возможно, в нашей моделе бытсрее происходит inference для сэмплинга

```
CUSTOM GRU CELL
DONE EPOCH 0, AVG LOSS 2.1978992079094932, TIME 827.3081226348877
DONE EPOCH 1, AVG LOSS 1.5335701902911718, TIME 826.921665430069
DONE EPOCH 2, AVG LOSS 1.4430663428760127, TIME 827.1765003204346
DONE EPOCH 3, AVG LOSS 1.3984494707578252, TIME 827.0133855342865
DONE EPOCH 4, AVG LOSS 1.370269677326428, TIME 827.1113862991333
DONE EPOCH 5, AVG LOSS 1.3507033528429684, TIME 826.9562947750092```

```
TF GRU CELL
DONE EPOCH 0, AVG LOSS 2.271053379176515, TIME 839.5970721244812
DONE EPOCH 1, AVG LOSS 1.5493454257136445, TIME 838.909827709198
DONE EPOCH 2, AVG LOSS 1.4537357697450104, TIME 838.884211063385```